In [2]:
import pandas as pd
import requests
from itertools import combinations
from statistics import mean

In [4]:
df = pd.read_csv('Data/Uniprot_Reference/SH2_IPR000980_uniprot_reference.csv')
uniprot_id_list = df['UniProt ID'].tolist()



In [ ]:
for uniprot_id in uniprot_id_list:
    print(uniprot_id)
    # res = requests.get(f'http://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}').json() 
    genename = (df.loc[df['UniProt ID'] == uniprot_id, 'Gene']).tolist()[0]
    domain = []
    domain_dict = {}
    boundaries = df.loc[df['UniProt ID'] == uniprot_id, 'Interpro Domains'].tolist()[0]
    b = boundaries.split(';')
    for entry_1 in b:
        domname, ipr_id, start, end = entry_1.split(':')
        domain_dict[domname] = start + '-' + end
        dom_range = start + '-' + end
        domain.append(dom_range)

    range_comb = []
    if len(domain) > 1:
        for combo in combinations(domain, 2):
            range_comb.append(combo)

    for r in (range_comb):
        source = r[0]
        target = r[1]

        s = source.split('-')

        s1 = s[0]
        s2 = s[1]

        t = target.split('-')

        t1 = t[0]
        t2 = t[1]

        get_link = requests.get(f'https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}').json()
        urllink = (get_link[0]['paeDocUrl'])
        data = requests.get(urllink).json()

        PAE_for_domain_1 = []  #one view of domain-domain interface

        for k in range(len(data[0]['predicted_aligned_error'])):
            list_scores = []
            if k in range(int(s1) - 1, int(s2)):

                m = (data[0]['predicted_aligned_error'][k])

                for index in range(int(t1) - 1, int(t2)):
                    list_scores.append(m[index])

            if list_scores:
                pae_for_res = []
                for pae_val in list_scores:
                    if pae_val <= 10:
                        pae_for_res.append(pae_val)

                pae_per = (len(pae_for_res) / len(list_scores)) * 100
                PAE_for_domain_1.append(pae_per)

        print(genename, ':', r, ':', mean(PAE_for_domain_1))

        PAE_for_domain_2 = []  #second view pof the onterface

        for k in range(len(data[0]['predicted_aligned_error'])):
            list_scores = []
            if k in range(int(t1) - 1, int(t2)):

                m = (data[0]['predicted_aligned_error'][k])

                for index in range(int(s1) - 1, int(s2)):
                    list_scores.append(m[index])

            if list_scores:
                pae_for_res = []
                for pae_val in list_scores:
                    if pae_val <= 10:
                        pae_for_res.append(pae_val)

                pae_per = (len(pae_for_res) / len(list_scores)) * 100
                PAE_for_domain_2.append(pae_per)

        print(genename, ':', r, ':', mean(PAE_for_domain_2))
        print(r, '-', (mean(PAE_for_domain_1) + mean(PAE_for_domain_2)) / 2)
    print(domain_dict, '\n')
